# Goal

Find CyCOGs corresponding to genes that have been observed to be up-regulated by MED4 _Prochlorococcus_ in response to viral infection in a laboratory setting. Reference: [Lindell et al. (2007)](https://www.nature.com/articles/nature06130)

## Inputs:
- Supplemental Table of MED4 genes up-regulated in response to viral infection (from Lindell et al. (2007))
- CyCOG to MED4 gene mapping file

## Output
- Dataframe matching each up-regulated gene with its corresponding CyCOG
